In [1]:
import numpy as np
import time
X = np.ndfromtxt('images.csv', delimiter=',')
y_orig = np.ndfromtxt("labels.csv", delimiter=',', dtype=np.int8)
img_size = X.shape[1]
num_class = 10
y = np.zeros([len(y_orig), num_class])
y[np.arange(len(y_orig)), y_orig] = 1
num_train = int(len(y) * 0.8)
print("X={}  y={}  num_train={}  img_size={}  num_class={}".format(X.shape,y.shape,num_train,img_size,num_class))

X=(10000, 784)  y=(10000, 10)  num_train=8000  img_size=784  num_class=10


In [2]:
X_train = X[0:num_train, :]
X_test = X[num_train:-1,:]
y_train = y[0:num_train]
y_test = y[num_train:-1]
print("X_train={}  y_train={}  X_test={}  y_test={}".format(X_train.shape,y_train.shape,X_test.shape,y_test.shape))

X_train=(8000, 784)  y_train=(8000, 10)  X_test=(1999, 784)  y_test=(1999, 10)


In [3]:
def h_elementwise(theta, X):
    # phi=(10000, 10)
    phi = np.zeros([X.shape[0], theta.shape[1]])
    for i in range(X.shape[0]):
        # theta.T=(10, 784)  X[i,:]=(784,)  temp.shape=(10,)
        temp = np.matmul(np.transpose(theta), X[i, :])
        # np.amax(temp).shape = ()
        temp = temp - np.amax(temp)
        # temp=(10,)  temp2.shape=(10,)
        temp2 = np.exp(temp)
        phi[i, :] = temp2 / np.sum(temp2)
    return(phi)

theta = np.zeros([img_size, num_class])
tempw = np.matmul(np.transpose(theta), X[0, :])
npamax = np.amax(tempw)
tempwn = tempw - np.amax(tempw)
print("theta.T={}  X[i,:]={}  tempw={}  npamax={}  tempwn={}".format(theta.T.shape, X[0, :].shape, tempw.shape, npamax, tempwn.shape))

%timeit h_elementwise(theta, X)

theta.T=(10, 784)  X[i,:]=(784,)  tempw=(10,)  npamax=0.0  tempwn=(10,)
183 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
def h_vec(theta, X):
    # X=(10000, 784)  theta.shape=(784, 10)  eta=(10000, 10)
    eta = np.matmul(X, theta)
    temp = np.exp(eta - np.reshape(np.amax(eta, axis=1), [-1, 1]))
    # npamaxrs=(10000, 1)  temp=(10000, 10)  npsum=(10000,)  npsumrs=(10000, 1)
#    print("eta={}  npamaxrs={}  temp={}  npsum={}  npsumrs={}".format(eta.shape, (np.reshape(np.amax(eta, axis=1),\
#        [-1, 1])).shape, temp.shape, (np.sum(temp, axis=1)).shape, (np.reshape(np.sum(temp, axis=1), [-1, 1])).shape))
    return (temp / np.reshape(np.sum(temp, axis=1), [-1, 1]))

%timeit h_vec(theta, X)

7.67 ms ± 53.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


With softmax as the activation function, for each observation $k$, we assign probabilities $p(y_j^{(k)}|x^{(k)})\equiv h_{j}^{(k)}$ for each class $j$:

$$\begin{align*}
p(y_j^{(k)}|x^{(k)})\equiv h_{j}^{(k)}\equiv\frac{e^{\theta_j^Tx^{(k)}}}{\sum_{q=1}^{10}e^{\theta_q^Tx^{(k)}}}
\end{align*}$$

Notice that $\theta$ is matrix with $n=784$ rows and $c=10$ columns.

Now we implement gradient descent, which was derived in the class notes 1, p.5:

$$\begin{align*}
\theta_{i,j}\mapsto\theta_{i,j}-\alpha\sum_{k=1}^m(h_{j}^{(k)}-y_j^{(k)})x_i^{(k)}
\tag{sftmx.1}
\end{align*}$$

Define

$$\begin{align*}
h_\theta(x^{(k)})=\begin{bmatrix}\frac{e^{\theta_1^Tx^{(k)}}}{\sum_{q=1}^{10}e^{\theta_q^Tx^{(k)}}}&\dots& \frac{e^{\theta_{10}^Tx^{(k)}}}{\sum_{q=1}^{10}e^{\theta_q^Tx^{(k)}}}\end{bmatrix}\quad y^{(k)}=\begin{bmatrix}0&\dots&1&\dots&0\end{bmatrix}
\end{align*}$$

and

$$\begin{align*}
h_\theta(X)=\begin{bmatrix}h_\theta(x^{(1)})\\\vdots\\h_\theta(x^{(m)})\end{bmatrix}=\begin{bmatrix}\frac{e^{\theta_1^Tx^{(1)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(1)}}}&\dots& \frac{e^{\theta_{10}^Tx^{(1)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(1)}}}\\\vdots&\ddots&\vdots\\\frac{e^{\theta_1^Tx^{(m)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(m)}}}&\dots& \frac{e^{\theta_{10}^Tx^{(m)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(m)}}}\end{bmatrix}\quad Y=\begin{bmatrix}y^{(1)}\\\vdots\\y^{(m)}\end{bmatrix}=\begin{bmatrix}0&\dots&1&\dots&0\\\vdots&\ddots&\vdots&\ddots&\vdots\\0&\dots&1&\dots&0\end{bmatrix}
\end{align*}$$

and

$$\begin{align*}
D&=h_\theta(X)-Y=\begin{bmatrix}h_\theta(x^{(1)})-y^{(1)}\\\vdots\\h_\theta(x^{(m)})-y^{(m)}\end{bmatrix}=\begin{bmatrix}\frac{e^{\theta_1^Tx^{(1)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(1)}}}-y_1^{(1)}&\dots& \frac{e^{\theta_{10}^Tx^{(1)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(1)}}}-y_{10}^{(1)}\\\vdots&\ddots&\vdots\\\frac{e^{\theta_1^Tx^{(m)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(m)}}}-y_{1}^{(m)}&\dots& \frac{e^{\theta_{10}^Tx^{(m)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(m)}}}-y_{10}^{(m)}\end{bmatrix}=\begin{bmatrix}h_{1}^{(1)}-y_1^{(1)}&\dots& h_{10}^{(1)}-y_{10}^{(1)}\\\vdots&\ddots&\vdots\\h_{1}^{(m)}-y_{1}^{(m)}&\dots& h_{10}^{(m)}-y_{10}^{(m)}\end{bmatrix}
\end{align*}$$

Hence

$$\begin{align*}
X^TD&=\begin{bmatrix}x_{1}^{(1)}&\dots&x_{1}^{(m)}\\\vdots&\ddots&\vdots\\x_{n}^{(1)}&\dots&x_{n}^{(m)}\end{bmatrix}\begin{bmatrix}\frac{e^{\theta_1^Tx^{(1)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(1)}}}-y_1^{(1)}&\dots& \frac{e^{\theta_{10}^Tx^{(1)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(1)}}}-y_{10}^{(1)}\\\vdots&\ddots&\vdots\\\frac{e^{\theta_1^Tx^{(m)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(m)}}}-y_{1}^{(m)}&\dots& \frac{e^{\theta_{10}^Tx^{(m)}}}{\sum_{k=1}^{10}e^{\theta_k^Tx^{(m)}}}-y_{10}^{(m)}\end{bmatrix}\\
    &=\begin{bmatrix}x_{1}^{(1)}&\dots&x_{1}^{(m)}\\\vdots&\ddots&\vdots\\x_{n}^{(1)}&\dots&x_{n}^{(m)}\end{bmatrix}_{n\times m}\begin{bmatrix}h_{1}^{(1)}-y_1^{(1)}&\dots& h_{10}^{(1)}-y_{10}^{(1)}\\\vdots&\ddots&\vdots\\h_{1}^{(m)}-y_{1}^{(m)}&\dots& h_{10}^{(m)}-y_{10}^{(m)}\end{bmatrix}_{m\times 10}\\
    &=\begin{bmatrix}\sum_{k=1}^{m}(h_{1}^{(k)}-y_1^{(k)})x_{1}^{(k)}&\dots&\sum_{k=1}^{m}(h_{10}^{(k)}-y_{10}^{(k)})x_{1}^{(k)}\\\vdots&\ddots&\vdots\\\sum_{k=1}^{m}(h_{1}^{(k)}-y_1^{(k)})x_{n}^{(k)}&\dots&\sum_{k=1}^{m}(h_{10}^{(k)}-y_{10}^{(k)})x_{n}^{(k)}\end{bmatrix}_{n\times 10}
\end{align*}$$

Then to update $\theta{i,j}$ for all $i,j$ as in sftmx.1, we have

$$\begin{align*}
\theta\mapsto\theta-\alpha X^TD
\end{align*}$$

In [7]:
# not fully vectorized
def GD(theta, X_train, y_train, alpha):
    # X_train=(8000, 784)  theta=(784,10)  diff=(8000,10)  y_train=(8000, 10)
    diff = h_vec(theta, X_train) - y_train
    #print("X_train={}  theta={}  diff={} y_train={}".format(X_train.shape, theta.shape, diff.shape, y_train.shape))
    for k in range(num_class):
        # diffT=(1,8000)  w=(1,784)
        diffT = np.reshape(diff[:, k], [1, -1])
        w = np.matmul(diffT, X_train)
        #if k==0: print("diffT={}  w={}".format(diffT.shape, w.shape))
        theta[:, k] -= alpha * np.squeeze(w)
        
def train(X_train, y_train, max_iter, alpha):
    theta = np.zeros([img_size, 10])
    for i in range(max_iter):
        GD(theta, X_train, y_train, alpha)       
    return theta

max_iter = 2
alpha = 0.001
start = time.time()
theta = train(X_train, y_train, max_iter, alpha)
end = time.time()
print("time elapsed: {} seconds   theta={}  X_test={}".format(end - start, theta.shape, X_test.shape))
pred = np.argmax(h_vec(theta, X_test), axis=1)
print("percentage correct: {0}".format(np.sum(pred == np.argmax(y_test, axis=1)) / float(len(y_test))))

time elapsed: 0.05970191955566406 seconds   theta=(784, 10)  X_test=(1999, 784)
percentage correct: 0.33016508254127064


In [6]:
#full vectorized
def GD_vec(theta, X_train, y_train, alpha):
    theta -= alpha * np.matmul(np.transpose(X_train), (h_vec(theta, X_train) - y_train))
    
def train_vec(X_train, y_train, max_iter, alpha):
    theta = np.zeros([img_size, 10])
    for i in range(max_iter):
        GD_vec(theta, X_train, y_train, alpha)       
    return theta

max_iter = 100
alpha = 0.001
start = time.time()
theta = train_vec(X_train, y_train, max_iter, alpha)
end = time.time()
print("time elapsed: {0} seconds".format(end - start))
pred = np.argmax(h_vec(theta, X_test), axis=1)
print("percentage correct: {0}".format(np.sum(pred == np.argmax(y_test, axis=1)) / float(len(y_test))))

time elapsed: 1.0092706680297852 seconds
percentage correct: 0.9259629814907454
